In [56]:
# Импорт библиотек
import datetime
import requests
import json
from pybit import spot
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from pybit.exceptions import InvalidRequestError
import numpy as np
from settings import *
from Def import *
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [57]:
# Получаем данные с интервалом
symbol = "BTCUSD"
interval = "1"
fields = "open_time,open,high,low,close,volume"

# Время
now = datetime.datetime.now()
start_of_today = datetime.datetime(now.year, now.month, now.day)

# Задаем количество дней, которые нужно взять для анализа
days_before_start = 6
days_before_end = 4
days = days_before_start - days_before_end

# Начальное и конечное время для запроса данных
start_time = str(int((start_of_today - datetime.timedelta(days=days_before_start)).timestamp()))
end_time = str(int((start_of_today - datetime.timedelta(days=days_before_end)).timestamp()))

# Запрос первой порции данных
url = f"https://api.bybit.com/v2/public/kline/list?symbol={symbol}&interval={interval}&from={start_time}&to={end_time}&limit=200&fields={fields}"
response = requests.get(url, headers={"api-key": API, "sign": secret})
data = json.loads(response.text)
df = pd.DataFrame(data['result'])

# Запрос оставшихся данных
while len(data['result']) == 200:
    start_time = str(int(data['result'][-1]['open_time']) + 60)  # Новое начальное время
    url = f"https://api.bybit.com/v2/public/kline/list?symbol={symbol}&interval={interval}&from={start_time}&to={end_time}&limit=200&fields={fields}"
    response = requests.get(url, headers={"api-key": API, "sign": secret})
    data = json.loads(response.text)
    df = pd.concat([df, pd.DataFrame(data['result'])], axis=0, ignore_index=True)

# Преобразуем время в формат datetime
df['open_time'] = pd.to_datetime(df['open_time'], unit='s')

# Устанавливаем индекс по времени
df = df.set_index('open_time')

# Удаляем столбец "symbol"
df = df.drop('symbol', axis=1)
df = df.drop('turnover', axis=1)
df = df.drop('interval', axis=1)

# Преобразуем значения столбцов в числа с плавающей точкой и удаляем дубликаты
df = df.astype(float)
df.drop_duplicates(inplace=True)

#Создаём столбцы с ADX
df['ADX'],df['DI+'],df['DI-'] = adx(df['high'], df['low'], df['close'], n=14)
df.dropna(inplace=True)

In [58]:
# Создаем столбцы с предыдущими значениями ADX, DI+ и DI-, low
df['prev_adx'] = df['ADX'].shift()
df['prev_di_plus'] = df['DI+'].shift()
df['prev2_di_minus'] = df['DI-'].shift()
df['prev_low'] = df['low'].shift()
df['diff_adx'] = ((df['ADX'] - df['ADX'].shift(2)) + (df['ADX'] - df['ADX'].shift()))/2
top_adx = 0
lowest_adx = 0
top_DI_pos= 0
#Удаляем Nan
df.dropna(inplace=True)
# Устанавливаем начальное значение флага позиции
position = 'out'

# Создаем список для хранения сигналов
signals = []
s=0
# Проходим по строкам таблицы
for index, row in df.iterrows():
    if row['ADX']>20 and row['ADX']>row['prev_adx']:
        top_adx=row['ADX']
    elif row['ADX']<20:
        top_adx=0

    # Условия покупки
    if position == 'out':
        if row['DI+'] - row['DI-'] > 80:
            s+=1
            print(s)
            print(row['diff_adx'])
            signals.append('buy')
            position = 'long'
        else:
            signals.append('hold')

    # Условия продажи
    elif position == 'long':
        if top_adx - row['ADX']>15:
            signals.append('sell')
            position = 'out'
        elif row['DI-'] > row['DI+']:
            signals.append('sell')
            position = 'out'
        else:
            signals.append('hold')

# Создаем столбец с сигналами
df['signal'] = signals

In [59]:
# Устанавливаем начальную сумму денег и количество криптовалют, которые у нас есть
initial_cash = 1000
initial_cash2 = initial_cash
initial_coins = 0

# Создаем список для хранения изменений стоимости криптовалют
coin_prices = []

# Устанавливаем комиссию в 0.1%
commission = 0.0001

# Инициализируем счетчики для статистики
total_trades = 0
successful_trades = 0
unsuccessful_trades = 0

# Проходим по строкам таблицы
for index, row in df.iterrows():
    # Если есть позиция на покупку
    if position == 'long':
        # Проверяем, если есть сигнал на продажу
        if row['signal'] == 'sell':
            # Продаем криптовалюты
            coins_sold = initial_coins
            coin_prices.append(row['close'])
            initial_cash += coins_sold * row['close'] * (1 - commission)
            initial_coins = 0
            position = 'out'

            # Обновляем счетчики статистики
            total_trades += 1
            if len(coin_prices) > 1 and coin_prices[-1] > coin_prices[-2]:
                successful_trades += 1
            else:
                unsuccessful_trades += 1

    # Если нет позиции на покупку, но есть сигнал на покупку
    elif position == 'out' and row['signal'] == 'buy':
        # Покупаем криптовалюты
        coins_bought = (initial_cash * (1 - commission)) / row['close']
        coin_prices.append(row['close'])
        initial_coins += coins_bought
        initial_cash = 0
        position = 'long'

# Если остались криптовалюты в конце, продаем их по последней известной цене
if initial_coins > 0:
    coins_sold = initial_coins
    coin_prices.append(df.iloc[-1]['close'])
    initial_cash += coins_sold * df.iloc[-1]['close'] * (1 - commission)
    initial_coins = 0
    position = 'out'

# Вычисляем изменение суммы денег
profit = initial_cash - initial_cash2

# Вычисляем win rate
win_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0

# Выводим результаты
print("initial_cash2:", initial_cash2)
print("Profit: $", profit)
print("Total Trades:", total_trades)
print("Successful Trades:", successful_trades)
print("Unsuccessful Trades:", unsuccessful_trades)
print("Win Rate: {:.2f}%".format(win_rate))
print("commission:",commission)
print("days:",days)

initial_cash2: 1000
Profit: $ 0
Total Trades: 0
Successful Trades: 0
Unsuccessful Trades: 0
Win Rate: 0.00%
commission: 0.0001
days: 2


In [60]:
# Создаем объект субграфика
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# Строим график свечей
candlestick = go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='candlestick')
fig.add_trace(candlestick, row=1, col=1)

# Строим график объема
bar = go.Bar(x=df.index,
             y=df['volume'],
             name='volume',
             marker=dict(color=df['close'] - df['open'],
                         colorscale='RdYlGn',
                         line=dict(color='gray', width=0.5)),
             opacity=0.8)
fig.add_trace(bar, row=2, col=1)

# Строим график ADX
adx = go.Scatter(x=df.index,
                 y=df['ADX'],
                 name='ADX',
                 line=dict(color='blue', width=1),
                 opacity=0.8)
fig.add_trace(adx, row=3, col=1)

# Строим график DI+
dip = go.Scatter(x=df.index,
                 y=df['DI+'],
                 name='DI+',
                 line=dict(color='green', width=1),
                 opacity=0.8)
fig.add_trace(dip, row=3, col=1)

# Строим график DI-
din = go.Scatter(x=df.index,
                 y=df['DI-'],
                 name='DI-',
                 line=dict(color='red', width=1),
                 opacity=0.8)
fig.add_trace(din, row=3, col=1)

# Создаем списки с координатами сигналов покупки и продажи
buy_signals = df.loc[df['signal'] == 'buy']
sell_signals = df.loc[df['signal'] == 'sell']

# Строим график сигналов покупки и продажи
if not buy_signals.empty:
    buy_scatter = go.Scatter(x=buy_signals.index, y=buy_signals['close'],
                             name='buy', mode='markers', marker=dict(color='green', size=10))
    fig.add_trace(buy_scatter, row=1, col=1)
if not sell_signals.empty:
    sell_scatter = go.Scatter(x=sell_signals.index, y=sell_signals['close'],
                              name='sell', mode='markers', marker=dict(color='red', size=10))
    fig.add_trace(sell_scatter, row=1, col=1)

# Настраиваем макет графика
fig.update_layout(title='BTCUSD',
                  yaxis_title='Price, USD',
                  height=1000,
                  xaxis_rangeslider_visible=True)

# Отображаем график
fig.show()